In [1]:
from data.process import *

2024-07-24 08:48:08.297623: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 08:48:08.307344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 08:48:08.321560: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 08:48:08.321594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 08:48:08.331349: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after first optimization
    'colsample_bytree': 0.25,
    'learning_rate': 0.1,

    # fixed after third optimization
    'max_depth': 30,
    'num_leaves': 2048,
    'min_data_in_leaf': 112,

    # override fixed params
    'n_estimators': 10000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['category']).columns)

In [6]:
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=100)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[100]	valid_0's auc: 0.88806	valid_0's binary_logloss: 0.245268
[200]	valid_0's auc: 0.889818	valid_0's binary_logloss: 0.243077
[300]	valid_0's auc: 0.890607	valid_0's binary_logloss: 0.242358
[400]	valid_0's auc: 0.891139	valid_0's binary_logloss: 0.241896
[500]	valid_0's auc: 0.891326	valid_0's binary_logloss: 0.241737
[600]	valid_0's auc: 0.891416	valid_0's binary_logloss: 0.241654
[700]	valid_0's auc: 0.89158	valid_0's binary_logloss: 0.241557
[800]	valid_0's auc: 0.891537	valid_0's binary_logloss: 0.241631


In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.8916411155254613),
                                     ('binary_logloss', 0.24149521517674621)])})


In [8]:
lgbm_output = model.predict_proba(test_df, iteration_range=(0, model._best_iteration + 1))[:, 1]